#Data Bootcamp Selection Challenge
#####In this challenge you will calculate various KPIs using a car based dataset, each question will have a single correct answer that will be evaluated through automated unit testing. Use the dictionary provided below to fill in your answers, each question will state the format required for the answer and examples are provided so you know how properly fill the answer dictionary. 
#####**Use the dataset "as is" and do not perform any data cleaning or modify it in any way, doing so could make you answer all your questions incorrectly. Do not modify the structure of the answer dictionary.**

#####When you finish this challenge please upload both your notebook and your answer dictionary in pickle format to a public github repository submit their URL to the [google form](https://forms.gle/wWysZEMkoZsjB11Y7) that was provided to you.

##### Some unit tests are provided at the end of this notebook to help you verify your answers are in the correct format, however they will not test everything.





In [1]:
#Use this dictionary to store your answers in the correct format in the cells below , do not modify the keys
answer_dict =  {"Q1" : None,
                "Q2" : None,
                "Q3" : None,
                "Q4" : None,
                "Q5" : None,
                "Q6" : None,
                "Q7" : None}

##Reading the dataset
#####An example is provided to read the dataset using [pandas](https://pandas.pydata.org/), while we reccommend using pandas you may use any python library to solve this challenge. 

In [2]:
import pandas as pd
import numpy as np
url='https://drive.google.com/file/d/1PCJ7ltluquoXKi6MYTPMfwZQNI_-MIFP/view?usp=sharing'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
df_pd = pd.read_csv(url)

In [3]:
#df_pd.head()

In [4]:
#df_pd.info()

## Installing PySpark

Downloading Java

In [5]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

Getting Spark and Hadoop

In [6]:
#!wget -q https://www-us.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz

Setting environment paths

In [7]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

Installing findspark

In [8]:
!pip install -q findspark
import findspark
findspark.init()

Finally able to import spark

In [9]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

Magic to make "show" pretty

In [10]:
%%html
<style>
.output_subarea.output_text.output_stream.output_stdout > pre {
  width:max-content;
}
.p-Widget.jp-RenderedText.jp-OutputArea-output > pre {
  width:max-content;
}


## Initial show

In [11]:
df = spark.createDataFrame(df_pd)
df.show(10, truncate=False)

+----------------+-------------------+----+-------------------+---------+---------------+-----------------+---------------------------+---------+------------------+--------+-----------+------------+-----------------------+--------------+
|Make            |Model              |Year|Engine Displacement|Cylinders|Transmission   |Drivetrain       |Vehicle Class              |Fuel Type|Fuel Barrels/Year |City MPG|Highway MPG|Combined MPG|CO2 Emission Grams/Mile|Fuel Cost/Year|
+----------------+-------------------+----+-------------------+---------+---------------+-----------------+---------------------------+---------+------------------+--------+-----------+------------+-----------------------+--------------+
|AM General      |DJ Po Vehicle 2WD  |1984|2.5                |4.0      |Automatic 3-spd|2-Wheel Drive    |Special Purpose Vehicle 2WD|Regular  |19.388823529411766|18      |17         |17          |522.7647058823529      |1950          |
|AM General      |FJ8c Post Office   |1984|4.2  

## General exploring

### Schema

In [12]:
df.printSchema()

root
 |-- Make: string (nullable = true)
 |-- Model: string (nullable = true)
 |-- Year: long (nullable = true)
 |-- Engine Displacement: double (nullable = true)
 |-- Cylinders: double (nullable = true)
 |-- Transmission: string (nullable = true)
 |-- Drivetrain: string (nullable = true)
 |-- Vehicle Class: string (nullable = true)
 |-- Fuel Type: string (nullable = true)
 |-- Fuel Barrels/Year: double (nullable = true)
 |-- City MPG: long (nullable = true)
 |-- Highway MPG: long (nullable = true)
 |-- Combined MPG: long (nullable = true)
 |-- CO2 Emission Grams/Mile: double (nullable = true)
 |-- Fuel Cost/Year: long (nullable = true)



### General info

In [13]:
df.describe().show()

+-------+----------+------------------+------------------+-------------------+-----------------+------------+----------------+--------------------+--------------------+------------------+------------------+------------------+-----------------+-----------------------+------------------+
|summary|      Make|             Model|              Year|Engine Displacement|        Cylinders|Transmission|      Drivetrain|       Vehicle Class|           Fuel Type| Fuel Barrels/Year|          City MPG|       Highway MPG|     Combined MPG|CO2 Emission Grams/Mile|    Fuel Cost/Year|
+-------+----------+------------------+------------------+-------------------+-----------------+------------+----------------+--------------------+--------------------+------------------+------------------+------------------+-----------------+-----------------------+------------------+
|  count|     35952|             35952|             35952|              35952|            35952|       35952|           35952|             

### Brands

In [14]:
df.select("Make").distinct().show(truncate=False)

+----------------------+
|Make                  |
+----------------------+
|Bertone               |
|Volkswagen            |
|Oldsmobile            |
|Sterling              |
|PAS, Inc              |
|Infiniti              |
|Peugeot               |
|Lambda Control Systems|
|Quantum Technologies  |
|J.K. Motors           |
|Pininfarina           |
|Lexus                 |
|Jaguar                |
|Roush Performance     |
|Evans Automobiles     |
|Saturn                |
|Ruf Automobile Gmbh   |
|Maserati              |
|Rolls-Royce           |
|Scion                 |
+----------------------+
only showing top 20 rows



## Imports

In [15]:
from pyspark.sql import functions as f
from pyspark.sql.window import Window
from pyspark.sql.types import FloatType

##Q1. What is the average CO2 emmission per gram/mile of all Volkswagen cars?

##### Format: A floating number
##### Example answer:
 `11.547`

Getting CO2 average emmissions

In [16]:
co2_mean_volks_df = df.where(df["Make"] == "Volkswagen").groupBy("Make").avg("CO2 Emission Grams/Mile")

Saving answer

In [17]:
co2_mean_volks_df.show()
answer_dict["Q1"] = co2_mean_volks_df.collect()[0][1]
print("Average CO2 emissions for Volkswagen -> {} gram/mile".format(answer_dict["Q1"]))

+----------+----------------------------+
|      Make|avg(CO2 Emission Grams/Mile)|
+----------+----------------------------+
|Volkswagen|           392.7417210857633|
+----------+----------------------------+

Average CO2 emissions for Volkswagen -> 392.7417210857633 gram/mile


##Q2. Calculate the top 5 brands(Make) with the most unique models, order your answer in descending order with respect to the number of unique models.
##### **NOTE:** Consider only the name of the models and their brand, that is use only the Make and Model columns
##### Format: A 5X2 list with each row being the name of the brand followed by the unique number of models, in descending order.
#####Hint: You can use the pandas [df.values.tolist()](https://pandas.pydata.org/docs/reference/api/pandas.Series.tolist.html) function to format your answer.

##### Example answer: 
`[["Volkswagen", 1000], ["Toyota", 900], ["Honda", 800], ["Subaru", 700], ["Ford", 600]]`

Aggregation to count unique models by brand

In [18]:
unique_models_brands_df = df.select("Make", "Model").groupBy("Make").agg(f.countDistinct("Model").alias("unique_models"))

Order by most number of unique models

In [19]:
unique_models_brands_df.orderBy(f.desc("unique_models")).show(10, truncate=False)


+-------------+-------------+
|Make         |unique_models|
+-------------+-------------+
|Mercedes-Benz|333          |
|BMW          |284          |
|Chevrolet    |253          |
|Ford         |185          |
|GMC          |163          |
|Porsche      |121          |
|Dodge        |118          |
|Toyota       |100          |
|Audi         |95           |
|Nissan       |94           |
+-------------+-------------+
only showing top 10 rows



Saving answer

In [20]:
# Collecting as a list of the first 5 Rows
top_5_um_brands_ls = unique_models_brands_df.orderBy(f.desc("unique_models")).head(5)
# Using list comprehension to "unpack" the results
answer_dict["Q2"] = [[top_5_um_brands_ls[n][0], top_5_um_brands_ls[n][1]] for n in range(5)]
print("Top 5 brands with the most unique models -> {}".format(answer_dict["Q2"]))

Top 5 brands with the most unique models -> [['Mercedes-Benz', 333], ['BMW', 284], ['Chevrolet', 253], ['Ford', 185], ['GMC', 163]]


##Q3. What are all the different types of fuels in the dataset sorted alphabetically?
##### Format: A list of strings sorted alphabetically.
##### Example Answer: 
`['Regular',
 'Premium']`

Getting the diferent types of fuels

In [21]:
fuel_types_df = df.select("Fuel Type").distinct().sort("Fuel Type")
fuel_types_df.show(truncate=False)

+---------------------------+
|Fuel Type                  |
+---------------------------+
|CNG                        |
|Diesel                     |
|Gasoline or E85            |
|Gasoline or natural gas    |
|Gasoline or propane        |
|Midgrade                   |
|Premium                    |
|Premium Gas or Electricity |
|Premium and Electricity    |
|Premium or E85             |
|Regular                    |
|Regular Gas and Electricity|
|Regular Gas or Electricity |
+---------------------------+



This is the answer using the dataset "as is". To avoid the case-sensitive sorting we could add a lowercase version of the *Fuel Type* column, sort by that column and drop it at the end.

In [22]:
# Collecting as a list of Rows
ordered_ft_ls = fuel_types_df.collect()
# Unpacking answer
answer_dict["Q3"] = [fuel_type[0] for fuel_type in ordered_ft_ls]
print("List of fuel types sorted alphabetically -> {}".format(answer_dict["Q3"]))

List of fuel types sorted alphabetically -> ['CNG', 'Diesel', 'Gasoline or E85', 'Gasoline or natural gas', 'Gasoline or propane', 'Midgrade', 'Premium', 'Premium Gas or Electricity', 'Premium and Electricity', 'Premium or E85', 'Regular', 'Regular Gas and Electricity', 'Regular Gas or Electricity']


##Q4. Show the 9 Toyota cars with the most extreme Fuel Barrels/Year in abosolute terms within all Toyota cars. Show the car Model, Year and their Fuel Barrels/Year in standard deviation units([Z-score](https://fredclavel.org/2019/03/18/basics-standardization-and-the-z-score/)) **sorted** in descending order by their Fuel Barrels/Year in absolute terms first and then by year in descending order **BUT** without modifying the negative values (see example).

##### Format: A 9X3 list with each row containing the Model, Year and Fuel Barrels/Year in standard deviations units

##### Example answer: 
```
[['DJ Po Vehicle 2WD', 2004, -6.407431084026927],
 ['FJ8c Post Office', 2003, -6.407431084026927],
 ['Post Office DJ5 2WD', 2005, -6.391684618442447],
 ['Sierra 2500 Hd 2WD', 2002, -6.391684618442447],
 ['Camry CNG', 2012, 2.677633075759575],
 ['Sierra 1500 4WD', 2005, 2.677633075759575],
 ['Sierra 1500 4WD', 2001, 2.677633075759575],
 ['V15 Suburban 4WD', 1988, 2.677633075759575],
 ['V15 Suburban 4WD', 1987, 2.677633075759575]]
```
#####Note that while the list is sorted by the Fuel Barrels/Year in absolute terms and in standard deviation units, the values are not modified. If the values are the same the rows are sorted by the year.


Calculating: sample's mean and standard deviation for Toyota cars

In [23]:
toyota_df = df.select("Make", "Model", "Year", "Fuel Barrels/Year").where(df["Make"] == "Toyota")
toyota_df.groupBy("Make").agg(f.mean("Fuel Barrels/Year"), f.stddev("Fuel Barrels/Year")).show()

+------+----------------------+------------------------------+
|  Make|avg(Fuel Barrels/Year)|stddev_samp(Fuel Barrels/Year)|
+------+----------------------+------------------------------+
|Toyota|    16.153677357417322|             4.087129593247344|
+------+----------------------+------------------------------+



In [24]:
toyota_s_mean, toyota_s_stddev = toyota_df.groupBy("Make").agg(f.mean("Fuel Barrels/Year"), 
                                                               f.stddev("Fuel Barrels/Year")).collect()[0][1:]
print("Toyota cars: mean -> {}, standard deviation -> {}".format(toyota_s_mean, toyota_s_stddev))

Toyota cars: mean -> 16.153677357417322, standard deviation -> 4.087129593247344


Obtaning Z-score for Fuel Barrels/Year

In [25]:
# Adding an extra column with the absolute value of the Z score
toyota_z_df = toyota_df.withColumn("z_score", (f.col("Fuel Barrels/Year") - toyota_s_mean)/toyota_s_stddev).withColumn("abs_z_score", f.abs("z_score"))
toyota_z_df.show(10)

+------+---------------+----+------------------+--------------------+-------------------+
|  Make|          Model|Year| Fuel Barrels/Year|             z_score|        abs_z_score|
+------+---------------+----+------------------+--------------------+-------------------+
|Toyota|1-Ton Truck 2WD|1985|15.695714285714288|-0.11205004912486004|0.11205004912486004|
|Toyota|1-Ton Truck 2WD|1985|15.695714285714288|-0.11205004912486004|0.11205004912486004|
|Toyota|1-Ton Truck 2WD|1986|15.695714285714288|-0.11205004912486004|0.11205004912486004|
|Toyota|1-Ton Truck 2WD|1986|14.330869565217393|-0.44598727557147405|0.44598727557147405|
|Toyota|1-Ton Truck 2WD|1987|15.695714285714288|-0.11205004912486004|0.11205004912486004|
|Toyota|1-Ton Truck 2WD|1987|14.330869565217393|-0.44598727557147405|0.44598727557147405|
|Toyota|1-Ton Truck 2WD|1988|           16.4805| 0.07996385608194201|0.07996385608194201|
|Toyota|1-Ton Truck 2WD|1988|18.311666666666667|  0.5279963015644824| 0.5279963015644824|
|Toyota|1-

Sorting "in descending order by their Fuel Barrels/Year in absolute terms first and then by year in descending order"

In [26]:
toyota_z_df.orderBy(f.desc("abs_z_score"), f.desc("Year")).show(10, truncate=False)

+------+---------------+----+------------------+-------------------+------------------+
|Make  |Model          |Year|Fuel Barrels/Year |z_score            |abs_z_score       |
+------+---------------+----+------------------+-------------------+------------------+
|Toyota|Cab/Chassis 2WD|1993|32.961            |4.112255865424802  |4.112255865424802 |
|Toyota|Cab/Chassis 2WD|1992|32.961            |4.112255865424802  |4.112255865424802 |
|Toyota|Cab/Chassis 2WD|1991|32.961            |4.112255865424802  |4.112255865424802 |
|Toyota|Cab/Chassis 2WD|1990|32.961            |4.112255865424802  |4.112255865424802 |
|Toyota|Cab/Chassis 2WD|1989|32.961            |4.112255865424802  |4.112255865424802 |
|Toyota|Camry CNG      |1999|0.0808695652173913|-3.9325417571185004|3.9325417571185004|
|Toyota|Camry CNG      |2001|0.0845454545454545|-3.931642375475663 |3.931642375475663 |
|Toyota|Camry CNG      |2000|0.0845454545454545|-3.931642375475663 |3.931642375475663 |
|Toyota|Cab/Chassis 2WD|1993|29.

Saving the answer

In [27]:
model_col, year_col, zscore_col = 1, 2, 4
# Limits the result to the top 9
q4_results_ls = toyota_z_df.orderBy(f.desc("abs_z_score"), f.desc("Year")).head(9)
# Unpacking answer
answer_dict["Q4"] = [[result[model_col], result[year_col], result[zscore_col]] for result in q4_results_ls]
print("Most extreme Fuel Barrels/Year Toyota cars -> {}x{}:{}".format(len(answer_dict["Q4"]), len(answer_dict["Q4"][0]), answer_dict["Q4"]))

Most extreme Fuel Barrels/Year Toyota cars -> 9x3:[['Cab/Chassis 2WD', 1993, 4.112255865424802], ['Cab/Chassis 2WD', 1992, 4.112255865424802], ['Cab/Chassis 2WD', 1991, 4.112255865424802], ['Cab/Chassis 2WD', 1990, 4.112255865424802], ['Cab/Chassis 2WD', 1989, 4.112255865424802], ['Camry CNG', 1999, -3.9325417571185004], ['Camry CNG', 2001, -3.931642375475663], ['Camry CNG', 2000, -3.931642375475663], ['Cab/Chassis 2WD', 1993, 3.3791118637261017]]


##Q5. Calculate the changes in Combined MPG with their previous model of all Golf cars with Manual 5-spd transmission and Regular Fuel Type. Show the Year, the Combined MPG and the calculated difference of MPG in a list sorted by Year in ascending order.

##### Format: A 19X3 list, with the Year and Combined MPG being of type integer **and only the calculated difference is of type float**
##### **Note: The value for the first model should be 0.** It does not matter that there are gaps in the years, calculate with respect the previous model.

#####Example answer:



```
[[1986, 25, 0.0],
 [1987, 25, 0.0],
 [1988, 25, 0.0],
 [1989, 25, 0.0],
 [1990, 23, -2.0],
 [1991, 23, 0.0],
 [1992, 24, 1.0],
 [1993, 25, 1.0],
 [1994, 25, 0.0],
 [1995, 25, 0.0],
 [1996, 25, 0.0],
 [1997, 25, 0.0],
 [1998, 24, -1.0],
 [1999, 25, 1.0],
 [2000, 24, -1.0],
 [2001, 24, 0.0],
 [2002, 24, 0.0],
 [2004, 24, 0.0],
 [2006, 24, 0.0]]
```





Preparing the dataset

In [28]:
diff_golf_df = df.select("Model", "Year", "Transmission", "Fuel Type", "Combined MPG").where((df["Model"] == "Golf") & (df["Transmission"] == "Manual 5-spd") & (df["Fuel Type"] == "Regular"))
diff_golf_df.show()

+-----+----+------------+---------+------------+
|Model|Year|Transmission|Fuel Type|Combined MPG|
+-----+----+------------+---------+------------+
| Golf|1986|Manual 5-spd|  Regular|          26|
| Golf|1987|Manual 5-spd|  Regular|          26|
| Golf|1988|Manual 5-spd|  Regular|          25|
| Golf|1989|Manual 5-spd|  Regular|          25|
| Golf|1999|Manual 5-spd|  Regular|          24|
| Golf|2000|Manual 5-spd|  Regular|          24|
| Golf|2001|Manual 5-spd|  Regular|          24|
| Golf|2002|Manual 5-spd|  Regular|          24|
| Golf|2003|Manual 5-spd|  Regular|          24|
| Golf|2004|Manual 5-spd|  Regular|          24|
| Golf|2005|Manual 5-spd|  Regular|          24|
| Golf|2006|Manual 5-spd|  Regular|          24|
| Golf|2010|Manual 5-spd|  Regular|          25|
| Golf|2011|Manual 5-spd|  Regular|          26|
| Golf|2012|Manual 5-spd|  Regular|          26|
| Golf|2013|Manual 5-spd|  Regular|          26|
| Golf|2015|Manual 5-spd|  Regular|          30|
| Golf|2016|Manual 5

In [29]:
# Defining a Window to process the previous row values
window = Window.partitionBy().orderBy("Year")
diff_golf_df = diff_golf_df.withColumn("previous_Combined_MPG", f.lag(df["Combined MPG"]).over(window))
# Calculating the difference, setting nulls as 0
diff_golf_df = diff_golf_df.withColumn("difference",
                                       diff_golf_df["Combined MPG"] - diff_golf_df["previous_Combined_MPG"])
diff_golf_df = diff_golf_df.na.fill(0)
# Select desired info and cast to float
q5_result_df = diff_golf_df.select("Year", "Combined MPG", diff_golf_df["difference"].cast(FloatType()))

Saving the answer

In [30]:
q5_result_df.show(5)

+----+------------+----------+
|Year|Combined MPG|difference|
+----+------------+----------+
|1986|          26|       0.0|
|1987|          26|       0.0|
|1988|          25|      -1.0|
|1989|          25|       0.0|
|1999|          24|      -1.0|
+----+------------+----------+
only showing top 5 rows



In [31]:
year_col, cmpg_col, diff_col = range(3)
q5_result_ls = q5_result_df.collect()
# Unpacking answer
answer_dict["Q5"] = [[result[year_col], result[cmpg_col], result[diff_col]] for result in q5_result_ls]
print("Changes in Combined MPG for specific Golf models -> {}x{}:{}".format(len(answer_dict["Q5"]), len(answer_dict["Q5"][0]), answer_dict["Q5"]))

Changes in Combined MPG for specific Golf models -> 19x3:[[1986, 26, 0.0], [1987, 26, 0.0], [1988, 25, -1.0], [1989, 25, 0.0], [1999, 24, -1.0], [2000, 24, 0.0], [2001, 24, 0.0], [2002, 24, 0.0], [2003, 24, 0.0], [2004, 24, 0.0], [2005, 24, 0.0], [2006, 24, 0.0], [2010, 25, 1.0], [2011, 26, 1.0], [2012, 26, 0.0], [2013, 26, 0.0], [2015, 30, 4.0], [2016, 30, 0.0], [2017, 29, -1.0]]


##Q6. What are the top 5 lowest CO2 Emission Grams/Mile emmisions of cars for each of the following brands: Toyota, Ford, Volkswagen, Nissan, Honda

#####Format: A 5X6 list with the first element of each row being the Make of the cars and the following five values being floats sorted in ascending order. The Makes should appear in order listed in the question starting with Toyota and ending with Honda (see example).

#####Example answer:

```
[['Toyota', 100.0, 140.0, 140.0, 150.0, 150.0],
 ['Ford',
  100.025641025641,
  200.677633075759575,
  200.677633075759575,
  200.677633075759575,
  200.677633075759575],
 ['Volkswagen', 139.0, 154.0, 166.5, 166.5, 166.5],
 ['Nissan', 122.0, 122.0, 122.0, 122.0, 160.0],
 ['Honda', 100.0, 100.0, 100.0, 100.0, 123.91684618442447]]
```






Preparing dataset

In [32]:
# Filtering
brands = ["Toyota", "Ford", "Volkswagen", "Nissan", "Honda"]
top_5_low_df = df.select("Make", "CO2 Emission Grams/Mile").where(df["Make"].isin(brands))
# Sorting
q6_result_df = top_5_low_df.orderBy("Make",f.asc("CO2 Emission Grams/Mile"))
for brand in brands:
  q6_result_df.where(f.col("Make") == brand).show(5)

+------+-----------------------+
|  Make|CO2 Emission Grams/Mile|
+------+-----------------------+
|Toyota|                  133.0|
|Toyota|                  133.0|
|Toyota|                  133.0|
|Toyota|                  133.0|
|Toyota|                  158.0|
+------+-----------------------+
only showing top 5 rows

+----+-----------------------+
|Make|CO2 Emission Grams/Mile|
+----+-----------------------+
|Ford|                  112.0|
|Ford|                  129.0|
|Ford|                  129.0|
|Ford|                  129.0|
|Ford|                  129.0|
+----+-----------------------+
only showing top 5 rows

+----------+-----------------------+
|      Make|CO2 Emission Grams/Mile|
+----------+-----------------------+
|Volkswagen|                  200.0|
|Volkswagen|                  200.0|
|Volkswagen|                  200.0|
|Volkswagen|                  200.0|
|Volkswagen|       261.025641025641|
+----------+-----------------------+
only showing top 5 rows

+------+--------

Saving answer

In [33]:
# Saving in an auxiliary dict to enforce correct order
q6_result_dic = {brand : q6_result_df.where(f.col("Make") == brand).head(5) for brand in brands}

In [34]:
# Column related to CO2 Emission Grams/Mile, i.e. q6_result_dic["Toyota"][0][co2em_col] -> 133.0
co2em_col = 1
# Initializing our brands list of lists -> [['Toyota'], ['Ford'], ['Volkswagen'], ['Nissan'], ['Honda']]
q6_result_ls = [[brand] for brand in brands]
for n, brand in enumerate(brands):
  # as our q6_result_dic has lists of rows as values, list comprehension is used to pivot through and unpack it
  q6_result_ls[n].extend( [q6_result_dic[brand][x][co2em_col] for x in range( len(q6_result_dic[brand]) ) ] )
# Unpacking answer
answer_dict["Q6"] = q6_result_ls
print("Top 5 lowest CO2 Emission Grams/Mile for brands: {} -> {}x{}:{}".format(brands, len(answer_dict["Q6"]), len(answer_dict["Q6"][0]), answer_dict["Q6"]))

Top 5 lowest CO2 Emission Grams/Mile for brands: ['Toyota', 'Ford', 'Volkswagen', 'Nissan', 'Honda'] -> 5x6:[['Toyota', 133.0, 133.0, 133.0, 133.0, 158.0], ['Ford', 112.0, 129.0, 129.0, 129.0, 129.0], ['Volkswagen', 200.0, 200.0, 200.0, 200.0, 261.025641025641], ['Nissan', 249.0, 254.0, 254.5, 254.5, 254.5], ['Honda', 130.0, 167.67924528301887, 167.67924528301887, 167.67924528301887, 167.67924528301887]]


##Q7. Form 7 groups of 5 years to calculated the median Combined MPG of each group. The first group is from 1984 to 1988, the second from 1989 to 1993 and so on. The last group will have years not appearing in the dataset.

#####Note: The groups ranges are inclusive on both sides, the first group starts with 1984 and cars from 1984 are included in it.
#####Format : A 7X2 list with the first element of each row being a tuple of two integers being the lower and uppper range of the year groups and the esecond element being the median Combined MPG of that group, a float number.

#####Example answer:


```
[[(1984, 1988), 11.0],
 [(1989, 1993), 10.0],
 [(1994, 1998), 10.0],
 [(1999, 2003), 14.0],
 [(2004, 2008), 13.0],
 [(2009, 2013), 14.0],
 [(2014, 2018), 15.0]]
```



Preparing datasets

In [35]:
# List of groups
groups_tp = [(group, group + 4) for group in range(1984, 2018, 5)]
# Creating a list of filtered Dataframes
grouped_df_ls = []
for group in groups_tp:
  grouped_df_ls.append(
      df.select("Year", "Combined MPG").where((f.col("Year") >= group[0]) &
                                              (f.col("Year") <= group[1])
                                              ).withColumn("group", f.lit(str(group)))
      )
# Showing the number of cars for each group
for n, group in enumerate(grouped_df_ls):
  print("Cars in group {}: {}".format(groups_tp[n], group.count()))

Cars in group (1984, 1988): 5731
Cars in group (1989, 1993): 5501
Cars in group (1994, 1998): 4208
Cars in group (1999, 2003): 4552
Cars in group (2004, 2008): 5678
Cars in group (2009, 2013): 5722
Cars in group (2014, 2018): 4560


In [36]:
# Putting back the DF
og_grouped_df = grouped_df_ls[0]
for piece_df in grouped_df_ls[1:]:
  og_grouped_df = og_grouped_df.union(piece_df)

Processing

In [37]:
# Defining a Window to rank each group
window = Window.partitionBy("group").orderBy("Combined MPG")
ranked_df = og_grouped_df.withColumn("row_id", f.row_number().over(window)).select("group", "row_id",  "Combined MPG")
# Showing first 3 values for each group
ranked_df.where(f.col("row_id") <= 3).show(25)

+------------+------+------------+
|       group|row_id|Combined MPG|
+------------+------+------------+
|(1984, 1988)|     1|           7|
|(1984, 1988)|     2|           7|
|(1984, 1988)|     3|           7|
|(2014, 2018)|     1|          10|
|(2014, 2018)|     2|          10|
|(2014, 2018)|     3|          11|
|(2004, 2008)|     1|           9|
|(2004, 2008)|     2|           9|
|(2004, 2008)|     3|           9|
|(2009, 2013)|     1|          10|
|(2009, 2013)|     2|          10|
|(2009, 2013)|     3|          10|
|(1989, 1993)|     1|           7|
|(1989, 1993)|     2|           7|
|(1989, 1993)|     3|           8|
|(1994, 1998)|     1|           8|
|(1994, 1998)|     2|           9|
|(1994, 1998)|     3|           9|
|(1999, 2003)|     1|           8|
|(1999, 2003)|     2|           9|
|(1999, 2003)|     3|           9|
+------------+------+------------+



In [38]:
# Collecting the number of registers by group in a list of Rows
count_group_rows_ls = ranked_df.groupBy("group").count().orderBy("group").collect()

In [39]:
# Parametrs to find the median
group_col, count_col, mid_index_col, odd_col, mean_value_col = range(5)
params_ls = [ [group[group_col], group[count_col], int(group[count_col]/2), group[count_col]%2 == 1, 0.0] for group in count_group_rows_ls]
params_ls

[['(1984, 1988)', 5731, 2865, True, 0.0],
 ['(1989, 1993)', 5501, 2750, True, 0.0],
 ['(1994, 1998)', 4208, 2104, False, 0.0],
 ['(1999, 2003)', 4552, 2276, False, 0.0],
 ['(2004, 2008)', 5678, 2839, False, 0.0],
 ['(2009, 2013)', 5722, 2861, False, 0.0],
 ['(2014, 2018)', 4560, 2280, False, 0.0]]

In [40]:
# Getting the data at the middle for each group
mid_data_ls = []
for param in params_ls:
  mid_data_ls.append(ranked_df.where((f.col("group") == param[group_col]) &
                                     ((f.col("row_id") == param[mid_index_col]) | (f.col("row_id") == param[mid_index_col] + 1))
                                     ).collect())
mid_data_ls

[[Row(group='(1984, 1988)', row_id=2865, Combined MPG=19),
  Row(group='(1984, 1988)', row_id=2866, Combined MPG=19)],
 [Row(group='(1989, 1993)', row_id=2750, Combined MPG=18),
  Row(group='(1989, 1993)', row_id=2751, Combined MPG=18)],
 [Row(group='(1994, 1998)', row_id=2104, Combined MPG=19),
  Row(group='(1994, 1998)', row_id=2105, Combined MPG=19)],
 [Row(group='(1999, 2003)', row_id=2276, Combined MPG=19),
  Row(group='(1999, 2003)', row_id=2277, Combined MPG=19)],
 [Row(group='(2004, 2008)', row_id=2839, Combined MPG=19),
  Row(group='(2004, 2008)', row_id=2840, Combined MPG=19)],
 [Row(group='(2009, 2013)', row_id=2861, Combined MPG=21),
  Row(group='(2009, 2013)', row_id=2862, Combined MPG=21)],
 [Row(group='(2014, 2018)', row_id=2280, Combined MPG=22),
  Row(group='(2014, 2018)', row_id=2281, Combined MPG=22)]]

In [41]:
# Middle indexes in mid_data_ls
first_half_col, second_half_col = range(2)
# Index of Combined MPG in mid_data_ls
combined_mpg_col = 2
for group, param in enumerate(params_ls):
  if param[odd_col]: # Odd number of registers in group
    param[mean_value_col] = mid_data_ls[group][second_half_col][combined_mpg_col] / 1.0
  else: # Even number of registers in group
    param[mean_value_col] = (mid_data_ls[group][first_half_col][combined_mpg_col] + mid_data_ls[group][second_half_col][combined_mpg_col]) / 2.0
params_ls

[['(1984, 1988)', 5731, 2865, True, 19.0],
 ['(1989, 1993)', 5501, 2750, True, 18.0],
 ['(1994, 1998)', 4208, 2104, False, 19.0],
 ['(1999, 2003)', 4552, 2276, False, 19.0],
 ['(2004, 2008)', 5678, 2839, False, 19.0],
 ['(2009, 2013)', 5722, 2861, False, 21.0],
 ['(2014, 2018)', 4560, 2280, False, 22.0]]

Saving answer

In [42]:
q6_result_ls = [ [group_tp, params_ls[index][mean_value_col]] for index, group_tp in enumerate(groups_tp)]
q6_result_ls

[[(1984, 1988), 19.0],
 [(1989, 1993), 18.0],
 [(1994, 1998), 19.0],
 [(1999, 2003), 19.0],
 [(2004, 2008), 19.0],
 [(2009, 2013), 21.0],
 [(2014, 2018), 22.0]]

In [43]:
# Saving answer in dic
answer_dict["Q7"] = [ [group_tp, params_ls[index][mean_value_col]] for index, group_tp in enumerate(groups_tp)]
print("Median Combined MPG of each group -> {}x{}:{}".format(len(answer_dict["Q7"]), len(answer_dict["Q7"][0]), answer_dict["Q7"]))

Median Combined MPG of each group -> 7x2:[[(1984, 1988), 19.0], [(1989, 1993), 18.0], [(1994, 1998), 19.0], [(1999, 2003), 19.0], [(2004, 2008), 19.0], [(2009, 2013), 21.0], [(2014, 2018), 22.0]]


##Test your answers

##### We provide you some tests to make sure your answer dictionary is in the correct format using unittest.
##### These tests are not meant to be comprehensive, you should review all your answers carefully.

In [44]:
import unittest

class TestAnswers(unittest.TestCase):
    def test_if_dict(self):
        self.assertIsInstance(answer_dict, dict)

    def test_keys(self):
        self.assertEqual(list(answer_dict.keys()), ['Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7'])

    def test_answers_types(self):
        types_values = [type(k) for k in answer_dict.values()]
        answer_types = [float, list, list, list, list, list, list]
        self.assertEqual(types_values, answer_types)

    def test_Q1(self):
        self.assertEqual(type(answer_dict['Q1']), float)

    def test_Q2_dim(self):
        self.assertEqual(np.array(answer_dict['Q2']).shape, (5,2))

    def test_Q2_types(self):
        dtype1 = type(answer_dict['Q2'][0][0])
        dtype2 = type(answer_dict['Q2'][0][1])
        self.assertEqual([dtype1, dtype2], [str, int])

    def test_Q3_types(self):
        q3_types = set([type(item) for item in answer_dict['Q3']])
        self.assertEqual(q3_types, {str})

    def test_Q4_dim(self):
        self.assertEqual(np.array(answer_dict['Q4']).shape, (9,3))

    def test_Q4_types(self):
        dtype1 = type(answer_dict['Q4'][0][0])
        dtype2 = type(answer_dict['Q4'][0][1])
        dtype3 = type(answer_dict['Q4'][0][2])
        self.assertEqual([dtype1, dtype2, dtype3], [str, int, float])

    def test_Q5_dim(self):
        self.assertEqual(np.array(answer_dict['Q5']).shape, (19,3))

    def test_Q5_types(self):
        dtype1 = type(answer_dict['Q5'][0][0])
        dtype2 = type(answer_dict['Q5'][0][1])
        dtype3 = type(answer_dict['Q5'][0][2])
        self.assertEqual([dtype1, dtype2, dtype3], [int, int, float])

    def test_Q5_first_zero(self):
        self.assertEqual(answer_dict['Q5'][0][2], 0)


    def test_Q6_dim(self):
        self.assertEqual(np.array(answer_dict['Q6']).shape, (5,6))

    def test_Q5_types(self):
        dtype1 = type(answer_dict['Q6'][0][0])
        dtype2 = type(answer_dict['Q6'][0][1])
        dtype3 = type(answer_dict['Q6'][0][2])
        dtype4 = type(answer_dict['Q6'][0][3])
        dtype5 = type(answer_dict['Q6'][0][4])
        dtype6 = type(answer_dict['Q6'][0][5])
        self.assertEqual([dtype1, dtype2, dtype3, dtype4, dtype5, dtype6], [str, float, float, float, float, float])

    def test_Q6_check_first_and_last_brand(self):
        first_brand = answer_dict['Q6'][0][0]
        last_brand = answer_dict['Q6'][4][0]

        self.assertEqual([first_brand, last_brand], ["Toyota", "Honda"])

    def test_Q7_dim(self):
        self.assertEqual(np.array(answer_dict['Q7'], dtype=object).shape, (7,2))

    def test_Q7_types(self):
        dtype1 = type(answer_dict['Q7'][0][0])
        dtype2 = type(answer_dict['Q7'][0][1])
        self.assertEqual([dtype1, dtype2], [tuple, float])

unittest.main(argv=[''], verbosity=2, exit=False)

test_Q1 (__main__.TestAnswers) ... ok
test_Q2_dim (__main__.TestAnswers) ... ok
test_Q2_types (__main__.TestAnswers) ... ok
test_Q3_types (__main__.TestAnswers) ... ok
test_Q4_dim (__main__.TestAnswers) ... ok
test_Q4_types (__main__.TestAnswers) ... ok
test_Q5_dim (__main__.TestAnswers) ... ok
test_Q5_first_zero (__main__.TestAnswers) ... ok
test_Q5_types (__main__.TestAnswers) ... ok
test_Q6_check_first_and_last_brand (__main__.TestAnswers) ... ok
test_Q6_dim (__main__.TestAnswers) ... ok
test_Q7_dim (__main__.TestAnswers) ... ok
test_Q7_types (__main__.TestAnswers) ... ok
test_answers_types (__main__.TestAnswers) ... ok
test_if_dict (__main__.TestAnswers) ... ok
test_keys (__main__.TestAnswers) ... ok

----------------------------------------------------------------------
Ran 16 tests in 0.050s

OK


##Save your answers


##### First, take a moment to evaluate your answers and make sure you have not missed anything

##### Use the following code to save your answers in pickle format, change the filename using the following format:
##### FIRSTNAME_LASTNAME_answers.pkl
##### Example: Juan_Perez_answers.pkl

##### If you are using google colab you can find your file on the left side bar by clicking the folder icon inside the sample_data folder. Remember to upload the pickle file and the notebook to github and submit their URLs to the [google form](https://forms.gle/wWysZEMkoZsjB11Y7).

In [45]:
answer_dict

{'Q1': 392.7417210857633,
 'Q2': [['Mercedes-Benz', 333],
  ['BMW', 284],
  ['Chevrolet', 253],
  ['Ford', 185],
  ['GMC', 163]],
 'Q3': ['CNG',
  'Diesel',
  'Gasoline or E85',
  'Gasoline or natural gas',
  'Gasoline or propane',
  'Midgrade',
  'Premium',
  'Premium Gas or Electricity',
  'Premium and Electricity',
  'Premium or E85',
  'Regular',
  'Regular Gas and Electricity',
  'Regular Gas or Electricity'],
 'Q4': [['Cab/Chassis 2WD', 1993, 4.112255865424802],
  ['Cab/Chassis 2WD', 1992, 4.112255865424802],
  ['Cab/Chassis 2WD', 1991, 4.112255865424802],
  ['Cab/Chassis 2WD', 1990, 4.112255865424802],
  ['Cab/Chassis 2WD', 1989, 4.112255865424802],
  ['Camry CNG', 1999, -3.9325417571185004],
  ['Camry CNG', 2001, -3.931642375475663],
  ['Camry CNG', 2000, -3.931642375475663],
  ['Cab/Chassis 2WD', 1993, 3.3791118637261017]],
 'Q5': [[1986, 26, 0.0],
  [1987, 26, 0.0],
  [1988, 25, -1.0],
  [1989, 25, 0.0],
  [1999, 24, -1.0],
  [2000, 24, 0.0],
  [2001, 24, 0.0],
  [2002, 24, 0

In [46]:
import pickle

file_name = "Irving_Galindo_answers.pkl"
path = ""

with open(path+file_name, 'wb') as f:
    pickle.dump(answer_dict, f, protocol=pickle.HIGHEST_PROTOCOL)